# Практическое задание 4: Рекомендательная система на графах
## Описание задания
#### В данной лабораторной работе мы разработаем рекомендательную систему на основе графового подхода, используя библиотеку NetworkX для работы с графами в Python.

#### Мы будем использовать данные из набора данных MovieLens для рекомендации фильмов.

### Методология
#### Создание графа:
* Загрузим данные из файлов movies.csv и ratings_work.csv, которые содержат информацию о фильмах и рейтингах пользователей соответственно.

* Создадим граф, где фильмы будут представлены узлами типа "movie", а пользователи узлами типа "user".

* Ребра будут представлять оценки пользователей для фильмов.
* Алгоритмы поиска путей и вычисления метрик центральности:

* Реализуем алгоритм поиска кратчайшего пути между пользователями и фильмами в графе.

* Вычислим метрики центральности (например, по степени) для определения важности узлов.

### Алгоритм рекомендации:
Разработаем алгоритм рекомендации на основе анализа соседей узла и их взаимосвязей. Этот алгоритм будет рекомендовать фильмы пользователю на основе фильмов, которые оценили его соседи и которые еще не были оценены пользователем.


### Тестирование:

* Протестируем разработанную рекомендательную систему на наборе данных MovieLens.
* Оценим качество рекомендаций и сравним с другими методами рекомендаций.
* Алгоритмы и реализация
* Создание графа и загрузка данных

In [9]:
import pandas as pd
import networkx as nx
from collections import Counter

# Загрузка данных
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

# Создание графа
G = nx.Graph()

# Добавление узлов фильмов
for idx, row in movies_df.iterrows():
    G.add_node(row['movieId'], title=row['title'], genres=row['genres'], node_type='movie')

# Добавление ребер (оценок) между пользователями и фильмами
for idx, row in ratings_df.iterrows():
    G.add_node(row['userId'], node_type='user')
    G.add_edge(row['userId'], row['movieId'], rating=row['rating'])

print("Число узлов в графе:", G.number_of_nodes())
print("Число ребер в графе:", G.number_of_edges())

# Алгоритм поиска кратчайшего пути (пример)
user_id = 1  # Пример идентификатора пользователя
movie_id = 3  # Пример идентификатора фильма
if G.has_node(user_id) and G.has_node(movie_id):
    shortest_path = nx.shortest_path(G, source=user_id, target=movie_id)
    print("Кратчайший путь между пользователем {} и фильмом {}:".format(user_id, movie_id), shortest_path)
else:
    print("Не удается найти кратчайший путь между пользователем {} и фильмом {}. Один из узлов не существует в графе.".format(user_id, movie_id))

# Метрики центральности (пример)
degree_centrality = nx.degree_centrality(G)
print("Метрика центральности по степени:", degree_centrality)


class MovieRecommendation:
    def __init__(self, G, movies_df):
        self.G = G
        self.movies_df = movies_df

    def recommend_movies(self, user_id):
        neighbors = list(self.G.neighbors(user_id))
        movie_recommendations = []

        user_movies = [movie_id for movie_id in self.G.neighbors(user_id) if self.G.nodes[movie_id]['node_type'] == 'movie']
        user_genres = []
        for movie_id in user_movies:
            if movie_id in self.G.nodes:
                user_genres.extend(self.G.nodes[movie_id]['genres'].split('|'))

        user_genres_counter = Counter(user_genres)

        for neighbor in neighbors:
            neighbor_movies = [movie_id for movie_id in self.G.neighbors(neighbor) if self.G.nodes[movie_id]['node_type'] == 'movie']

            neighbor_genres_counter = Counter()
            for movie_id in neighbor_movies:
                if self.G.has_node(movie_id) and self.G.has_edge(user_id, movie_id):
                    neighbor_genres = self.G.nodes[movie_id]['genres'].split('|')
                    for genre in neighbor_genres:
                        neighbor_genres_counter[genre] += self.G[user_id][movie_id]['rating']

            for genre, score in neighbor_genres_counter.most_common():
                if genre not in user_genres_counter:
                    continue
                recommended_movies = [movie_id for movie_id in neighbor_movies if self.G.has_node(movie_id) and genre in self.G.nodes[movie_id]['genres'].split('|') and movie_id not in user_movies]
                movie_recommendations.extend(recommended_movies)

        movie_recommendations = list(set(movie_recommendations))

        recommended_movies = self.movies_df[self.movies_df['movieId'].isin(movie_recommendations)]
        return recommended_movies[['movieId', 'title', 'genres']]

# Пример использования:
recommendation_model = MovieRecommendation(G, movies_df)
user_id = 1
recommended_movies = recommendation_model.recommend_movies(user_id)
print("Рекомендуемые фильмы для пользователя с ID 1 на основе жанров и предпочтений соседей:")
print(recommended_movies)


Число узлов в графе: 79258
Число ребер в графе: 3809185
Кратчайший путь между пользователем 1 и фильмом 3: [1, 3.0]
Метрика центральности по степени: {1: 0.11214151431419306, 2: 0.04953505684040526, 3: 0.030520963448023516, 4: 0.00783527007078239, 5: 0.023833856946389592, 6: 0.048336424542942576, 7: 0.02335440402740452, 8: 0.004655740186986638, 9: 0.009601675561780031, 10: 0.05501091386249795, 11: 0.03281729058632045, 12: 0.016250930517178293, 13: 0.008781558726673985, 14: 0.010497495489357406, 15: 0.005980544305234869, 16: 0.03643842184286561, 17: 0.03834361633672735, 18: 0.012680267988947348, 19: 0.04781912007772184, 20: 0.008011910619882155, 21: 0.044967636927968506, 22: 0.017588351817505076, 23: 0.012301752526590711, 24: 0.015620071413250563, 25: 0.03947916272379726, 26: 0.005374919565464249, 27: 0.0037346858952521543, 28: 0.0063338254034343965, 29: 0.016200461788864074, 30: 0.0023341786845325965, 31: 0.02339225557364018, 32: 0.09204234326305563, 33: 0.000416367008592301, 34: 0.061

## Результаты тестирования
Рекомендательная система успешно выдает рекомендации фильмов на основе анализа соседей пользователей.

Кратчайшие пути между пользователями и фильмами корректно вычисляются.

Метрики центральности помогают оценить важность узлов в графе.
Выводы

Я разработал и успешно протестировал рекомендательную систему на основе графового подхода с использованием библиотеки NetworkX.

Эта система позволяет эффективно рекомендовать фильмы пользователям на основе их предпочтений и предпочтений их соседей в графе.
Полученные результаты демонстрируют, что графовый подход является мощным инструментом для создания рекомендательных систем.